On Wednesday, 6th March, Chancellor of the Exchequer Jeremy Hunt will present the government's 2024 Spring Budget. Ahead of the announcement, media reports speculate that the Chancellor is considering cutting the main National Insurance rate, in an extension to his previous cuts in the Autumn Statement 2023 (analysed by PolicyEngine in [this report](https://policyengine.org/uk/research/autumn-statement-2023)).

In this post, we'll estimate the impacts of such a cut upon the budget, as well as upon society-wide economic outcomes. We'll also look at the impact these changes would have upon individual UK households.

## Budget reform options

Separate media sources identify four potential reforms that the Chancellor is considering:

* Cuts to Employee (Class 1) National Insurance Contributions (NICs) by either 1p[^1] or 2p[^3]
* A 2p cut to the basic rate of tax, lowering the rate from 20p to 18p[^4]
* A fuel duty freeze[^1]

PolicyEngine's top-level economic impact estimates follow below. In modelling these reforms, we assume each took effect on 1 January 2024 and will evaluate their impacts over calendar year 2024.

[^1]:
      [The Guardian, 2024](https://www.theguardian.com/politics/2024/mar/02/tax-and-spending-cuts-will-backfire-economists-warn-jeremy-hunt)
[^2]: 
      [Institute for Government, 2024](https://www.instituteforgovernment.org.uk/comment/five-things-look-out-spring-budget-2024)
[^3]:
      [Financial Times, 2024](https://www.ft.com/content/c2d56d45-c061-4d5a-bf4f-1eddd1fa7d25)
[^4]:
      [Financial Times, 2024](https://www.ft.com/content/1b4f19d7-c7e5-4d16-a7c1-1c12ec0d0592)

*Table 1: Potential budget reforms and their impacts*

In [16]:
from policyengine_uk.model_api import *
from policyengine_uk import Microsimulation
import pandas as pd

# DataFrame with columns: [provision name, provision cost in 2024, percent better off, provision poverty impact in 2024, inequality impact in 2024]

reforms = []

ni_rate_cut_1p = {
    "gov.hmrc.national_insurance.class_1.rates.employee.main": {
        "year:2024:5": 0.09,
    }
}

ni_rate_cut_2p = {
    "gov.hmrc.national_insurance.class_1.rates.employee.main": {
        "year:2024:5": 0.08,
    }
}

base_rate_cut_2p = {
    "gov.hmrc.income_tax.rates.uk[0].rate": {
        "year:2024:5": 0.18,
    }
}

fuel_duty_freeze = {
    "gov.hmrc.fuel_duty.petrol_and_diesel": {
        "year:2024:5": 0.5083
    }
}

reforms = [ni_rate_cut_1p, ni_rate_cut_2p, base_rate_cut_2p, fuel_duty_freeze]

provisions = ["1p NI cut", "2p NI cut", "2p basic rate cut", "Fuel duty cut"]

baseline = Microsimulation()

def get_impacts(reform_dict: dict) -> dict:
    reform = Reform.from_dict(reform_dict, country_id="uk")
    reformed = Microsimulation(reform=reform)

    budgetary_impact = baseline.calculate("household_net_income", period=2024).sum() - reformed.calculate("household_net_income", 2024).sum()
    poverty_baseline = baseline.calculate("in_poverty", map_to="person", period=2024).mean()
    poverty_reform = reformed.calculate("in_poverty", map_to="person", period=2024).mean()
    poverty_rate_change = (poverty_reform - poverty_baseline) / poverty_baseline

    gini_baseline = baseline.calculate("equiv_household_net_income", map_to="person", period=2024).gini()
    gini_reform = reformed.calculate("equiv_household_net_income", map_to="person", period=2024).gini()
    gini_rate_change = (gini_reform - gini_baseline) / gini_baseline

    hnet_reformed = reformed.calculate("household_net_income", period=2024, map_to="person")
    hnet_baseline = baseline.calculate("household_net_income", period=2024, map_to="person")
    percent_better_off = (hnet_reformed > hnet_baseline).mean()
    # Poverty headcount relative change
    # Inequality gini index relative change
    # Percent better off

    return dict(
        budgetary_impact=budgetary_impact,
        reform_id=reform.api_id,
        poverty_rate_change=poverty_rate_change,
        gini_rate_change=gini_rate_change,
        percent_better_off=percent_better_off
    )

budgetary_impacts = []
reform_ids = []
poverty_rate_changes = []
gini_rate_changes = []
percentages_better_off = []

for i in range(len(provisions)):
    impacts = get_impacts(reforms[i])
    budgetary_impacts.append(impacts["budgetary_impact"])
    reform_ids.append(impacts["reform_id"])
    poverty_rate_changes.append(impacts["poverty_rate_change"])
    gini_rate_changes.append(impacts["gini_rate_change"])
    percentages_better_off.append(impacts["percent_better_off"])

df = pd.DataFrame({
    "Reform": provisions,
    "Budgetary impact": budgetary_impacts,
    "Percent gaining": percentages_better_off,
    "Poverty change": poverty_rate_changes,
    "Inequality change": gini_rate_changes,
    "PolicyEngine": reform_ids,
})

df["Budgetary impact"] = df["Budgetary impact"].apply(lambda x: f"{x/1e9:+,.1f}")
df["PolicyEngine"] = df["PolicyEngine"].apply(lambda x: f"[#{x}](https://policyengine.org/uk/policy?reform={x}&baseline=1&time_period=2024&region=uk)")
df["Percent gaining"] =  df["Percent gaining"].apply(lambda x: f"{x:+.1%}")
df["Poverty change"] =  df["Poverty change"].apply(lambda x: f"{x:+.1%}")
df["Inequality change"] =  df["Inequality change"].apply(lambda x: f"{x:+.1%}")

In [20]:
from IPython.display import Markdown

Markdown(df.to_markdown(index=False))

| Reform            |   Budgetary impact | Percent gaining   | Poverty change   | Inequality change   | PolicyEngine                                                                                    |
|:------------------|-------------------:|:------------------|:-----------------|:--------------------|:------------------------------------------------------------------------------------------------|
| 1p NI cut         |               -5   | 63.8%             | -0.0%            | 0.2%                | [#49642](https://policyengine.org/uk/policy?reform=49642&baseline=1&time_period=2024&region=uk) |
| 2p NI cut         |              -10   | 63.8%             | -0.2%            | 0.3%                | [#49643](https://policyengine.org/uk/policy?reform=49643&baseline=1&time_period=2024&region=uk) |
| 2p basic rate cut |              -11.1 | 69.6%             | -0.4%            | 0.3%                | [#49644](https://policyengine.org/uk/policy?reform=49644&baseline=1&time_period=2024&region=uk) |
| Fuel duty cut     |               -0.4 | 42.7%             | -0.1%            | -0.0%               | [#49645](https://policyengine.org/uk/policy?reform=49645&baseline=1&time_period=2024&region=uk) |

## Distributional impacts

In [25]:
from policyengine_core.charts.api import get_api_chart_data
import plotly.express as px
from policyengine_core.charts import *

decile_df = pd.DataFrame()

for i in range(len(provisions)):
    impacts = get_impacts(reforms[i])
    decile_data = get_api_chart_data("uk", Reform.from_dict(reforms[i], "uk").api_id, "decile", "uk", 2024, 1)["relative"]
    for decile in range(1, 11):
        decile_df = decile_df.append({
            "Reform": provisions[i],
            "Decile": decile,
            "Relative change": decile_data[str(decile)]
        }, ignore_index=True)

fig = px.bar(
    decile_df,
    x="Decile",
    y="Relative change",
    color="Reform",
    barmode="group",
).update_layout(
    title="Change in net income by decile",
    xaxis_title="Reform",
    yaxis_title="Relative change in net income",
    yaxis_tickformat="+.1%",
)

fig = format_fig(fig)
fig

In [31]:
Markdown(f"""
## National Insurance

Turning first to the macro impacts, the PolicyEngine model projects that a reduction in the Class 1 NIC contribution rate by 2p costs the government approximately £{-float(df.iloc[1].values[1])} billion.
"""
         )


## National Insurance

Turning first to the macro impacts, the PolicyEngine model projects that a reduction in the Class 1 NIC contribution rate by 2p costs the government approximately £10.0 billion.


## National Insurance

Turning first to the macro impacts, the PolicyEngine model projects that a reduction in the Class 1 NIC contribution rate by 2p costs the government approximately £10.8 billion.

[chart]

Under this 2p cut, every income decile gains in aggregate, with 63p of households increasing their post-tax income, and none seeing a decrease in disposable. However, top earners do especially well. With a reduction by 2p in Class 1 NIC contribution rates, the average household in income decile 1 receives £4 more post-tax income, while the average household in decile 10 receives £1,087. Under a reduction by 1p, these values are halved, to £2 and £585, respectively.

[chart]

The NIC contribution reform generates an ambiguous effect upon income inequality - while the top 1% of the population's share of net income decreases, the Gini income inequality coefficient actually increases by 0.4p. The poverty rate also falls slightly (0.1 percentage points, or 0.3p against its current rate).

[charts]

Turning to the household level, reducing NIC contributions by 2p increases post-tax income for a single person with employment income if they earn at least £12,570, much like the reforms previously modelled by PolicyEngine following the Autumn Statement 2023. This individual's take-home pay would increase linearly until reaching a maximum benefit of £817 at an annual income of £50,270. Were NIC contributions to only be reduced by 1p, the income phase-in and phase-out points remain the same, but said individual's maximum benefit would be only £440 annually.

Mirroring the reforms from Autumn 2023, most individuals within the proscribed income range would see each 1p reduction in NIC contributions correspond with a 1p reduction in marginal tax rate. Individuals on the very ends of the range would see slightly lower reductions. [why is this?]

[charts]

# Basic Rate Cuts




In [3]:
import pandas as pd
import plotly.express as px
from policyengine_core.charts import *
from policyengine_uk import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
from plotly.express.colors import sample_colorscale


def modify_parameters(parameters):
    parameters.gov.hmrc.national_insurance.class_1.rates.employee.main.update(
        start=instant("2024-01-01"), stop=instant("2024-12-31"),
        value=0.08)
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
baseline_person = baseline.calc("household_net_income",
    period=2024, map_to="person")
reformed_person = reformed.calc("household_net_income",
    period=2024, map_to="person")
difference_person = reformed_person - baseline_person


# fig = format_fig(fig)
# 
# for i in range(len(fig.data)):
#     fig.data[i].marker.color = c[i]
# 
# fig.update_layout(
#     legend_title="Calendar year",
#     xaxis_tickvals=chosen_metrics,
#     xaxis_ticktext=["Child Benefit", "Housing Benefit", "Universal Credit"],
#     xaxis_title="",
#     yaxis_title="Budgetary impact (£)",
# )